In [111]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, AffinityPropagation, AgglomerativeClustering
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
# from config import db_user, db_password
import pymongo
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.figure_factory as ff
import boto3
import os


# AWS Connection

## Import Housing Dataset

In [2]:
# set variables for bucket, file name, and s3. 
bucket = "sleeper-cities"
file_name = "housing_data_realtor_cleaned.csv"
s3 = boto3.client('s3') 
# create connection to S3 using default config and all buckets within S3
obj = s3.get_object(Bucket= bucket, Key= file_name) 
# get object and file (key) from bucket
housing_df = pd.read_csv(obj['Body']) 
# Review the data 
housing_df.head()

,date,metro_area_city,median_listing_price,active_listing_count,median_days_on_market,new_listing_count,price_increased_count,price_reduced_count,pending_listing_count,average_listing_price,total_listing_count
0,6/1/21,"new york-newark-jersey city, ny-nj-pa",617500.0,61846,55.0,26280,1020,9540,32230,1.260094e+06,94076
1,6/1/21,"los angeles-long beach-anaheim, ca",1024500.0,13258,44.0,10096,476,1928,15494,2.507311e+06,28752
2,6/1/21,"chicago-naperville-elgin, il-in-wi",354950.0,19089,33.0,17088,444,5712,25296,5.380922e+05,44385
3,6/1/21,"dallas-fort worth-arlington, tx",387000.0,6850,29.0,9008,704,1924,14504,5.952097e+05,21354
4,6/1/21,"houston-the woodlands-sugar land, tx",365995.0,13440,36.0,11368,1748,3636,17155,5.565350e+05,30595


## Import Income & Population Dataset

In [3]:
# set variables for bucket, file name, and s3. 
bucket = "sleeper-cities"
file_name = "metro_names_income_pop.csv"
s3 = boto3.client('s3') 
# create connection to S3 using default config and all buckets within S3
obj = s3.get_object(Bucket= bucket, Key= file_name) 
# get object and file (key) from bucket
income_pop_df = pd.read_csv(obj['Body']) 
# Review the data 
income_pop_df.head()

,metro_area_city,income,pop
0,"aberdeen, sd",30802,28257
1,"aberdeen, wa",23090,16756
2,"abilene, tx",24529,123420
3,"ada, ok",23538,17235
4,"adrian, mi",20799,20600


## Cleaning Housing Data

In [4]:
# check the types from the dataframe
housing_df.dtypes

date                      object
metro_area_city           object
median_listing_price     float64
active_listing_count       int64
median_days_on_market    float64
new_listing_count          int64
price_increased_count      int64
price_reduced_count        int64
pending_listing_count      int64
average_listing_price    float64
total_listing_count        int64
dtype: object

In [5]:
# Convert Date to a datetime column
housing_df["date"] = pd.to_datetime(housing_df["date"])
housing_df.dtypes

date                     datetime64[ns]
metro_area_city                  object
median_listing_price            float64
active_listing_count              int64
median_days_on_market           float64
new_listing_count                 int64
price_increased_count             int64
price_reduced_count               int64
pending_listing_count             int64
average_listing_price           float64
total_listing_count               int64
dtype: object

In [6]:
# Display new DataFrame
housing_df.head()

,date,metro_area_city,median_listing_price,active_listing_count,median_days_on_market,new_listing_count,price_increased_count,price_reduced_count,pending_listing_count,average_listing_price,total_listing_count
0,2021-06-01,"new york-newark-jersey city, ny-nj-pa",617500.0,61846,55.0,26280,1020,9540,32230,1.260094e+06,94076
1,2021-06-01,"los angeles-long beach-anaheim, ca",1024500.0,13258,44.0,10096,476,1928,15494,2.507311e+06,28752
2,2021-06-01,"chicago-naperville-elgin, il-in-wi",354950.0,19089,33.0,17088,444,5712,25296,5.380922e+05,44385
3,2021-06-01,"dallas-fort worth-arlington, tx",387000.0,6850,29.0,9008,704,1924,14504,5.952097e+05,21354
4,2021-06-01,"houston-the woodlands-sugar land, tx",365995.0,13440,36.0,11368,1748,3636,17155,5.565350e+05,30595


In [7]:
# Extract the year from the 'date' column.
housing_df["year"] = pd.DatetimeIndex(housing_df['date']).year
housing_df.head()

,date,metro_area_city,median_listing_price,active_listing_count,median_days_on_market,new_listing_count,price_increased_count,price_reduced_count,pending_listing_count,average_listing_price,total_listing_count,year
0,2021-06-01,"new york-newark-jersey city, ny-nj-pa",617500.0,61846,55.0,26280,1020,9540,32230,1.260094e+06,94076,2021
1,2021-06-01,"los angeles-long beach-anaheim, ca",1024500.0,13258,44.0,10096,476,1928,15494,2.507311e+06,28752,2021
2,2021-06-01,"chicago-naperville-elgin, il-in-wi",354950.0,19089,33.0,17088,444,5712,25296,5.380922e+05,44385,2021
3,2021-06-01,"dallas-fort worth-arlington, tx",387000.0,6850,29.0,9008,704,1924,14504,5.952097e+05,21354,2021
4,2021-06-01,"houston-the woodlands-sugar land, tx",365995.0,13440,36.0,11368,1748,3636,17155,5.565350e+05,30595,2021


In [8]:
# Filter to show only data from 2019
housing_df_filtered = housing_df[housing_df['year'] == 2019]
print(housing_df_filtered.shape)
housing_df_filtered.head()

(11004, 12)


,date,metro_area_city,median_listing_price,active_listing_count,median_days_on_market,new_listing_count,price_increased_count,price_reduced_count,pending_listing_count,average_listing_price,total_listing_count,year
16506,2019-12-01,"new york-newark-jersey city, ny-nj-pa",549999.5,65994,83.5,10272,428,7132,16934,1.110110e+06,82928,2019
16507,2019-12-01,"los angeles-long beach-anaheim, ca",877500.0,17466,75.5,4632,300,1972,11449,2.112489e+06,28915,2019
16508,2019-12-01,"chicago-naperville-elgin, il-in-wi",299250.0,31563,68.5,7588,240,5440,12497,4.389860e+05,44060,2019
16509,2019-12-01,"dallas-fort worth-arlington, tx",335500.0,19493,64.5,5780,680,5240,9232,4.688301e+05,28725,2019
16510,2019-12-01,"houston-the woodlands-sugar land, tx",299994.0,26614,69.0,7032,836,5988,9069,4.375289e+05,35683,2019


In [9]:
# Group by metro_area/city - index split separately 
grouped_housing_df = housing_df_filtered.groupby(['metro_area_city'],axis=0,as_index=False).mean()
grouped_housing_df.head(270)

,metro_area_city,median_listing_price,active_listing_count,median_days_on_market,new_listing_count,price_increased_count,price_reduced_count,pending_listing_count,average_listing_price,total_listing_count,year
0,"aberdeen, sd",182846.428567,161.750000,81.333333,43.333333,0.000000,36.000000,0.333333,224213.729800,162.083333,2019.0
1,"aberdeen, wa",243286.190475,336.000000,71.375000,110.000000,4.333333,65.000000,125.666667,281836.496283,461.666667,2019.0
2,"abilene, tx",214853.273808,561.166667,66.416667,195.000000,3.333333,149.000000,282.000000,247640.667450,843.166667,2019.0
3,"ada, ok",155062.500008,151.666667,67.500000,44.333333,0.000000,20.666667,53.666667,192856.850308,205.333333,2019.0
4,"adrian, mi",169820.238100,428.416667,64.416667,123.333333,2.000000,83.666667,31.250000,221564.868992,459.666667,2019.0
...,...,...,...,...,...,...,...,...,...,...,...
265,"escanaba, mi",133711.904767,195.166667,124.583333,37.333333,0.000000,25.333333,43.583333,172138.761758,238.750000,2019.0
266,"espaòola, nm",280412.500000,151.083333,172.208333,17.666667,0.000000,16.000000,25.833333,525734.395083,176.916667,2019.0
267,"eugene, or",387748.571425,712.416667,50.291667,356.000000,24.333333,273.333333,108.000000,467880.311958,820.416667,2019.0
268,"eureka-arcata-fortuna, ca",377169.047617,366.583333,75.916667,120.666667,2.333333,87.000000,19.666667,529712.607350,386.250000,2019.0


In [34]:
# change the issue with espanola 
grouped_housing_df.at[266,'metro_area_city'] = "espanola, nm"
grouped_housing_df.head(270)

,metro_area_city,median_listing_price,active_listing_count,median_days_on_market,new_listing_count,price_increased_count,price_reduced_count,pending_listing_count,average_listing_price,total_listing_count,year
0,"aberdeen, sd",182846.428567,161.750000,81.333333,43.333333,0.000000,36.000000,0.333333,224213.729800,162.083333,2019.0
1,"aberdeen, wa",243286.190475,336.000000,71.375000,110.000000,4.333333,65.000000,125.666667,281836.496283,461.666667,2019.0
2,"abilene, tx",214853.273808,561.166667,66.416667,195.000000,3.333333,149.000000,282.000000,247640.667450,843.166667,2019.0
3,"ada, ok",155062.500008,151.666667,67.500000,44.333333,0.000000,20.666667,53.666667,192856.850308,205.333333,2019.0
4,"adrian, mi",169820.238100,428.416667,64.416667,123.333333,2.000000,83.666667,31.250000,221564.868992,459.666667,2019.0
...,...,...,...,...,...,...,...,...,...,...,...
265,"escanaba, mi",133711.904767,195.166667,124.583333,37.333333,0.000000,25.333333,43.583333,172138.761758,238.750000,2019.0
266,"espanola, nm",280412.500000,151.083333,172.208333,17.666667,0.000000,16.000000,25.833333,525734.395083,176.916667,2019.0
267,"eugene, or",387748.571425,712.416667,50.291667,356.000000,24.333333,273.333333,108.000000,467880.311958,820.416667,2019.0
268,"eureka-arcata-fortuna, ca",377169.047617,366.583333,75.916667,120.666667,2.333333,87.000000,19.666667,529712.607350,386.250000,2019.0


In [69]:
grouped_housing_df.head(330)

,metro_area_city,median_listing_price,active_listing_count,median_days_on_market,new_listing_count,price_increased_count,price_reduced_count,pending_listing_count,average_listing_price,total_listing_count,year
0,"aberdeen, sd",182846.428567,161.750000,81.333333,43.333333,0.000000,36.000000,0.333333,224213.729800,162.083333,2019.0
1,"aberdeen, wa",243286.190475,336.000000,71.375000,110.000000,4.333333,65.000000,125.666667,281836.496283,461.666667,2019.0
2,"abilene, tx",214853.273808,561.166667,66.416667,195.000000,3.333333,149.000000,282.000000,247640.667450,843.166667,2019.0
3,"ada, ok",155062.500008,151.666667,67.500000,44.333333,0.000000,20.666667,53.666667,192856.850308,205.333333,2019.0
4,"adrian, mi",169820.238100,428.416667,64.416667,123.333333,2.000000,83.666667,31.250000,221564.868992,459.666667,2019.0
...,...,...,...,...,...,...,...,...,...,...,...
325,"grand island, ne",200023.809517,189.583333,60.625000,66.666667,1.000000,39.000000,10.750000,246621.533575,200.333333,2019.0
326,"grand junction, co",348335.119050,797.166667,60.250000,325.333333,11.333333,247.333333,494.333333,657524.502550,1291.500000,2019.0
327,"grand rapids-wyoming, mi",287636.845233,2461.833333,48.875000,1291.333333,32.666667,833.333333,24.416667,363139.046208,2486.250000,2019.0
328,"grants pass, or",374051.190483,433.916667,77.875000,121.666667,1.333333,127.666667,10.250000,544014.552708,444.166667,2019.0


In [48]:
# Change the issue with canon
grouped_housing_df.at[142,'metro_area_city'] = "canon city, co"
grouped_housing_df.head(144)

,metro_area_city,median_listing_price,active_listing_count,median_days_on_market,new_listing_count,price_increased_count,price_reduced_count,pending_listing_count,average_listing_price,total_listing_count,year
0,"aberdeen, sd",182846.428567,161.750000,81.333333,43.333333,0.000000,36.000000,0.333333,224213.729800,162.083333,2019.0
1,"aberdeen, wa",243286.190475,336.000000,71.375000,110.000000,4.333333,65.000000,125.666667,281836.496283,461.666667,2019.0
2,"abilene, tx",214853.273808,561.166667,66.416667,195.000000,3.333333,149.000000,282.000000,247640.667450,843.166667,2019.0
3,"ada, ok",155062.500008,151.666667,67.500000,44.333333,0.000000,20.666667,53.666667,192856.850308,205.333333,2019.0
4,"adrian, mi",169820.238100,428.416667,64.416667,123.333333,2.000000,83.666667,31.250000,221564.868992,459.666667,2019.0
...,...,...,...,...,...,...,...,...,...,...,...
139,"carlsbad-artesia, nm",256956.035717,204.333333,54.250000,62.666667,2.000000,28.000000,18.916667,275349.304642,223.250000,2019.0
140,"carson city, nv",384663.625000,109.166667,49.500000,55.333333,0.666667,42.333333,103.750000,465558.274017,212.916667,2019.0
141,"casper, wy",239612.797608,293.250000,56.541667,111.000000,2.000000,66.000000,0.250000,286881.372300,293.500000,2019.0
142,"canon city, co",327886.279767,289.000000,67.375000,95.000000,1.000000,66.666667,98.333333,432917.235442,387.333333,2019.0


In [71]:
# Change the issue with canon
grouped_housing_df.at[327,'metro_area_city'] = "grand rapids, mi"
grouped_housing_df.head(330)

,metro_area_city,median_listing_price,active_listing_count,median_days_on_market,new_listing_count,price_increased_count,price_reduced_count,pending_listing_count,average_listing_price,total_listing_count,year
0,"aberdeen, sd",182846.428567,161.750000,81.333333,43.333333,0.000000,36.000000,0.333333,224213.729800,162.083333,2019.0
1,"aberdeen, wa",243286.190475,336.000000,71.375000,110.000000,4.333333,65.000000,125.666667,281836.496283,461.666667,2019.0
2,"abilene, tx",214853.273808,561.166667,66.416667,195.000000,3.333333,149.000000,282.000000,247640.667450,843.166667,2019.0
3,"ada, ok",155062.500008,151.666667,67.500000,44.333333,0.000000,20.666667,53.666667,192856.850308,205.333333,2019.0
4,"adrian, mi",169820.238100,428.416667,64.416667,123.333333,2.000000,83.666667,31.250000,221564.868992,459.666667,2019.0
...,...,...,...,...,...,...,...,...,...,...,...
325,"grand island, ne",200023.809517,189.583333,60.625000,66.666667,1.000000,39.000000,10.750000,246621.533575,200.333333,2019.0
326,"grand junction, co",348335.119050,797.166667,60.250000,325.333333,11.333333,247.333333,494.333333,657524.502550,1291.500000,2019.0
327,"grand rapids, mi",287636.845233,2461.833333,48.875000,1291.333333,32.666667,833.333333,24.416667,363139.046208,2486.250000,2019.0
328,"grants pass, or",374051.190483,433.916667,77.875000,121.666667,1.333333,127.666667,10.250000,544014.552708,444.166667,2019.0


In [49]:
# Create a breakpoint
breakpoint

<function breakpoint>

## Connect to AWS RDS 

In [3]:
import pandas as pd
from sqlalchemy import create_engine

# follows django database settings format, replace with your own settings
DATABASES = {
    'production':{
        'NAME': 'postgres',
        'USER': 'root',
        'PASSWORD': 'sleepercities4',
        'HOST': 'sleeper-cities-db.c5kluwhs2m0u.us-east-2.rds.amazonaws.com',
        'PORT': 5432,
    },
}

# choose the database to use
db = DATABASES['production']

# construct an engine connection string
engine_string = "postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}".format(
    user = db['USER'],
    password = db['PASSWORD'],
    host = db['HOST'],
    port = db['PORT'],
    database = db['NAME'],
)

# create sqlalchemy engine
engine = create_engine(engine_string)

## Export Data to Postgres

In [73]:
# Load housing dataframe into postgres
grouped_housing_df.to_sql('housing_data', engine,if_exists='replace',index=False, method='multi')

In [52]:
# Load income & population into postgres
income_pop_df.to_sql('income_population_data', engine,if_exists='replace',index=False, method='multi')

### Next Steps - Jump into Postgres and Join tables

## Load combined cleaned data from Postgress back into Panda Dataframes

In [4]:
# read a table from database into pandas dataframe, replace "tablename" with your table name
housing_income_pop_df = pd.read_sql_table('housing_income_pop_data',engine)
housing_income_pop_df.head(270)

,metro_area_city,median_listing_price,active_listing_count,median_days_on_market,new_listing_count,price_increased_count,price_reduced_count,pending_listing_count,average_listing_price,total_listing_count,year,income,pop
0,"aberdeen, sd",182846.428567,161.750000,81.333333,43.333333,0.000000,36.000000,0.333333,224213.729800,162.083333,2019.0,30802,28257
1,"aberdeen, wa",243286.190475,336.000000,71.375000,110.000000,4.333333,65.000000,125.666667,281836.496283,461.666667,2019.0,23090,16756
2,"abilene, tx",214853.273808,561.166667,66.416667,195.000000,3.333333,149.000000,282.000000,247640.667450,843.166667,2019.0,24529,123420
3,"ada, ok",155062.500008,151.666667,67.500000,44.333333,0.000000,20.666667,53.666667,192856.850308,205.333333,2019.0,23538,17235
4,"adrian, mi",169820.238100,428.416667,64.416667,123.333333,2.000000,83.666667,31.250000,221564.868992,459.666667,2019.0,20799,20600
...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,"escanaba, mi",133711.904767,195.166667,124.583333,37.333333,0.000000,25.333333,43.583333,172138.761758,238.750000,2019.0,23440,12160
266,"espanola, nm",280412.500000,151.083333,172.208333,17.666667,0.000000,16.000000,25.833333,525734.395083,176.916667,2019.0,21349,10044
267,"eugene, or",387748.571425,712.416667,50.291667,356.000000,24.333333,273.333333,108.000000,467880.311958,820.416667,2019.0,31021,172622
268,"eureka-arcata-fortuna, ca",377169.047617,366.583333,75.916667,120.666667,2.333333,87.000000,19.666667,529712.607350,386.250000,2019.0,25871,19133


# Prepare data for Machine Learning

### Note: Make sure we use the right table loaded from Postgres

In [5]:
# create new df with DF cities  
metro_name_df = pd.DataFrame(housing_income_pop_df['metro_area_city'])
print(metro_name_df.shape)
metro_name_df

(917, 1)


,metro_area_city
0,"aberdeen, sd"
1,"aberdeen, wa"
2,"abilene, tx"
3,"ada, ok"
4,"adrian, mi"
...,...
912,"youngstown-warren-boardman, oh-pa"
913,"yuba city, ca"
914,"yuma, az"
915,"zanesville, oh"


In [6]:
# create new DF with columns needed for ML - Drop: (year, price red, price inc, pending listing)
ml_housing_income_pop_df= housing_income_pop_df[['median_listing_price','active_listing_count','median_days_on_market','new_listing_count','average_listing_price','total_listing_count','income','pop']]
print(ml_housing_income_pop_df.shape)
ml_housing_income_pop_df.head()

(917, 8)


,median_listing_price,active_listing_count,median_days_on_market,new_listing_count,average_listing_price,total_listing_count,income,pop
0,182846.428567,161.750000,81.333333,43.333333,224213.729800,162.083333,30802,28257
1,243286.190475,336.000000,71.375000,110.000000,281836.496283,461.666667,23090,16756
2,214853.273808,561.166667,66.416667,195.000000,247640.667450,843.166667,24529,123420
3,155062.500008,151.666667,67.500000,44.333333,192856.850308,205.333333,23538,17235
4,169820.238100,428.416667,64.416667,123.333333,221564.868992,459.666667,20799,20600


In [7]:
# Standardize the data with StandardScaler().
X_scaled = StandardScaler().fit_transform(ml_housing_income_pop_df)
X_scaled[:5]

array([[-0.41134894, -0.26119984, -0.00626211, -0.26879905, -0.39071823,
        -0.27383375,  0.63668812, -0.25943154],
       [-0.01127082, -0.21946674, -0.4072039 , -0.22459811, -0.20049605,
        -0.22012706, -0.47037499, -0.3315778 ],
       [-0.19948115, -0.16553905, -0.60683601, -0.16824192, -0.31338208,
        -0.15173508, -0.26380551,  0.33753002],
       [-0.59526332, -0.26361481, -0.56321891, -0.26813604, -0.49423243,
        -0.26608027, -0.40606427, -0.32857301],
       [-0.49757518, -0.19733284, -0.68735988, -0.21575793, -0.39946256,
        -0.22048561, -0.79924967, -0.30746422]])

In [8]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)
pca.fit(X_scaled)
print(pca.explained_variance_ratio_)

X_pca = pca.transform(X_scaled)

[0.51048887 0.24226372 0.12547681]


In [9]:
# Create a DataFrame with the three principal components.
pcs_df = pd.DataFrame(X_pca, columns=['PC 1','PC 2', 'PC 3'])
pcs_df.head(10)

,PC 1,PC 2,PC 3
0,-0.499891,0.025396,-0.336139
1,-0.551533,0.103539,-0.358122
2,-0.186804,0.364752,-0.641400
3,-0.784764,0.575732,-0.614407
4,-0.738936,0.646516,-0.590835
5,0.474245,0.879584,-1.022092
6,-0.744040,0.334229,0.574511
7,-0.897690,1.032337,0.362880
8,0.286068,-0.874189,-1.239458
9,0.938669,0.205143,0.313412


In [178]:
# Create an elbow curve to find the best value for K.
inertia = []
k = list(range(1, 15))

# Calculate the inertia for the range of K values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)

# Create the elbow curve
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

C:\Users\ryan4\anaconda3\envs\mlenv\lib\site-packages\sklearn\cluster\_kmeans.py:882: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.



:Curve   [k]   (inertia)

## KMeans

In [179]:
# Initialize the K-Means model.
model = KMeans(n_clusters=7, random_state=0)

# Fit the model
model.fit(pcs_df)

# Predict clusters
prediction = model.predict(pcs_df)
print(prediction)

[3 3 3 3 3 3 0 0 5 5 0 3 0 4 3 5 0 3 3 3 3 0 3 0 3 0 5 3 3 5 3 5 0 3 0 3 5
 3 3 5 3 3 3 3 3 2 5 3 3 3 3 3 3 4 0 5 4 3 3 3 5 0 3 0 3 0 5 3 3 0 3 3 3 0
 3 0 3 5 3 5 0 0 0 0 3 0 5 3 5 5 3 3 3 3 0 0 0 0 5 3 0 3 4 5 3 5 0 3 0 1 5
 3 0 1 0 5 3 0 0 0 3 3 3 3 3 5 3 3 3 5 0 0 0 3 3 3 4 3 3 3 5 5 5 3 3 3 3 3
 3 3 3 3 0 5 4 5 5 5 2 5 3 4 0 3 0 3 3 0 3 4 3 3 3 5 0 3 3 5 3 5 3 5 3 3 4
 5 3 3 5 0 0 5 3 5 3 0 5 3 0 3 0 5 3 3 0 0 2 3 3 3 0 5 3 3 3 3 0 3 3 0 5 0
 4 3 5 4 5 3 3 3 0 3 0 0 3 3 3 0 3 5 3 5 3 0 3 5 3 1 3 3 3 0 3 0 3 0 3 0 5
 5 3 3 3 3 3 0 0 5 5 3 3 3 3 3 3 5 3 3 3 3 5 3 5 3 0 5 3 3 3 3 0 0 5 3 3 0
 3 0 3 3 3 3 5 3 3 3 5 0 3 3 5 3 0 0 3 1 0 3 3 3 0 1 0 3 3 3 5 5 5 0 0 3 5
 3 3 5 5 3 0 3 3 5 0 3 0 3 0 3 1 3 3 3 3 0 3 5 3 3 3 1 5 0 3 3 3 3 3 5 5 3
 3 5 3 5 3 0 0 2 0 3 3 3 5 3 0 3 3 5 0 4 3 3 5 0 3 3 3 0 3 1 4 0 3 0 3 3 3
 3 3 5 0 3 0 3 3 0 5 1 5 5 3 4 1 3 3 3 3 5 5 0 1 5 3 3 5 3 0 0 3 5 3 5 3 5
 3 3 3 3 3 5 3 3 3 3 3 3 3 0 4 0 0 5 3 3 3 3 3 0 3 5 3 0 3 5 0 3 0 5 3 0 3
 0 3 3 3 5 5 2 4 3 0 3 0 

In [180]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the ml_housing_df and pcs_df DataFrames on the same columns.
clustered_k_df = ml_housing_income_pop_df.join(pcs_df, how='inner')

#  Add a new column, "metro_area/city" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_k_df = clustered_k_df.join(metro_name_df, how='inner')

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
# Add the predicted class columns
clustered_k_df["Class"] = model.labels_
clustered_k_df.head(10)

# Print the shape of the clustered_df
print(clustered_k_df.shape)
clustered_k_df.head(10)

(917, 13)


,median_listing_price,active_listing_count,median_days_on_market,new_listing_count,average_listing_price,total_listing_count,income,pop,PC 1,PC 2,PC 3,metro_area_city,Class
0,182846.428567,161.750000,81.333333,43.333333,224213.729800,162.083333,30802,28257,-0.499891,0.025396,-0.336139,"aberdeen, sd",3
1,243286.190475,336.000000,71.375000,110.000000,281836.496283,461.666667,23090,16756,-0.551533,0.103539,-0.358122,"aberdeen, wa",3
2,214853.273808,561.166667,66.416667,195.000000,247640.667450,843.166667,24529,123420,-0.186804,0.364752,-0.641400,"abilene, tx",3
3,155062.500008,151.666667,67.500000,44.333333,192856.850308,205.333333,23538,17235,-0.784764,0.575732,-0.614407,"ada, ok",3
4,169820.238100,428.416667,64.416667,123.333333,221564.868992,459.666667,20799,20600,-0.738936,0.646516,-0.590835,"adrian, mi",3
5,165456.904758,1701.500000,54.166667,798.666667,244281.999525,2889.083333,24688,197597,0.474245,0.879584,-1.022092,"akron, oh",3
6,185736.904767,410.666667,99.833333,90.666667,220033.831333,514.833333,24999,31980,-0.744040,0.334229,0.574511,"alamogordo, nm",0
7,120405.726192,545.666667,91.500000,126.666667,150925.848925,563.250000,20313,72130,-0.897690,1.032337,0.362880,"albany, ga",0
8,349433.928575,450.583333,50.375000,232.333333,448459.134575,682.500000,29664,55338,0.286068,-0.874189,-1.239458,"albany, or",5
9,297317.107150,3856.000000,83.000000,1118.666667,342436.266492,6182.666667,26440,70295,0.938669,0.205143,0.313412,"albany-schenectady-troy, ny",5


In [181]:
# Find Austin metro area 
austin_group_k = clustered_k_df.loc[clustered_k_df['metro_area_city'].str.contains("austin", case=False)]
austin_group_k

,median_listing_price,active_listing_count,median_days_on_market,new_listing_count,average_listing_price,total_listing_count,income,pop,PC 1,PC 2,PC 3,metro_area_city,Class
52,155880.357150,124.083333,60.125000,42.000000,185615.222250,177.083333,27965,25233,-0.565727,0.374262,-1.054393,"austin, mn",3
53,359709.845233,7644.250000,56.708333,3252.666667,552108.749992,10896.666667,38904,556140,4.516994,0.150833,-0.777391,"austin-round rock, tx",4


In [182]:
# Metro_area/city with austin group
metro_area_city_k = clustered_k_df.loc[clustered_k_df['Class'] == 4]
print(f'Silhouette Score(n=5): {silhouette_score(pcs_df, prediction, metric="euclidean")}')
print(f'Variance Ratio Criterion: {calinski_harabasz_score(pcs_df, model.labels_)}')
print(f'Davies-Bouldin Index: {davies_bouldin_score(pcs_df, model.labels_)}')
print(f'Number of Cities: {metro_area_city_k.shape[0]}')
metro_area_city_k.head(10)

Silhouette Score(n=5): 0.3654034087448475
Variance Ratio Criterion: 680.0829635146803
Davies-Bouldin Index: 0.722012801824567
Number of Cities: 35


,median_listing_price,active_listing_count,median_days_on_market,new_listing_count,average_listing_price,total_listing_count,income,pop,PC 1,PC 2,PC 3,metro_area_city,Class
13,268651.244050,2470.083333,54.958333,1104.333333,350132.412267,4352.666667,30403,560513,2.158623,0.496140,-1.161531,"albuquerque, nm",4
53,359709.845233,7644.250000,56.708333,3252.666667,552108.749992,10896.666667,38904,556140,4.516994,0.150833,-0.777391,"austin-round rock, tx",4
56,323465.178567,9550.916667,55.833333,3853.000000,417196.827275,10743.333333,43368,249434,4.121117,0.189771,-0.838788,"baltimore-columbia-towson, md",4
102,583291.880958,9825.083333,51.208333,5195.333333,886703.358750,12665.000000,58838,299980,6.239407,-2.194391,-1.090491,"boston-cambridge-newton, ma-nh",4
136,303677.232142,9477.416667,86.375000,2283.333333,509095.152358,12299.750000,29980,140799,2.840199,0.459692,0.808653,"cape coral-fort myers, fl",4
154,341191.744050,8189.833333,57.833333,3566.333333,456165.700833,12507.500000,32563,353107,3.938183,0.660109,-0.416211,"charlotte-concord-gastonia, nc-sc",4
161,270155.732142,5471.500000,54.166667,2539.000000,354225.385358,9727.250000,30531,303940,2.724807,0.790292,-0.769175,"cincinnati, oh-ky-in",4
169,195869.404767,5874.916667,63.166667,2358.666667,285278.250925,9608.916667,22833,217383,1.967354,1.500810,-0.189533,"cleveland-elyria, oh",4
184,288182.738100,3827.916667,50.375000,2204.666667,363495.870667,6723.166667,29322,898553,3.731078,1.131323,-1.241460,"columbus, oh",4
222,504546.892858,7635.750000,41.291667,4697.666667,670959.989258,13907.416667,37628,421478,5.232733,-0.379517,-0.983159,"denver-aurora-lakewood, co",4


In [ ]:
# Export KMeans Cluster to .CSV
# metro_area_city_k.to_csv(r'../Resources/KMeans_output.csv', index = True)

### Kmeans Plots

In [163]:
# Creating a 3D-Scatter with the PCA data and the clusters
# Plot
fig = px.scatter_3d(clustered_k_df, x="PC 1", y="PC 2", z="PC 3", color="Class", symbol="Class",size="average_listing_price", size_max=25, width=800, hover_name="metro_area_city", hover_data=["average_listing_price"])
fig.update_layout(legend=dict(x=0,y=1))
fig.show()


In [167]:
# 2D scatter plot
clustered_k_df.hvplot.scatter(
    x="PC 1",
    y="PC 2",
    hover_cols=["metro_area_city"],
    by="Class",
)

:NdOverlay   [Class]
   :Scatter   [PC 1]   (PC 2,metro_area_city)

### Validating Cluster Model 

(Metric used to calculate the goodness of a clustering technique)

In [169]:
# Silhouette Score (calculated using the mean intra-cluster distance (a) and the mean nearest-cluster distance (b) for each sample.)
KMean= KMeans(n_clusters=6)
KMean.fit(pcs_df)
label=KMean.predict(pcs_df)
labels=KMean.labels_
print(f'Silhouette Score(n=6): {silhouette_score(pcs_df, label, metric="euclidean")}')

#print(f'Silhouette Samples(n=5): {silhouette_samples(pcs_df, label, metric="euclidean")}')

# Variance Ratio Criterion (Calinski-Harabasz Index: the ratio of the sum of between-clusters dispersion and of inter-cluster dispersion for all clusters)
print(f'Variance Ratio Criterion: {calinski_harabasz_score(pcs_df, labels)}')

# Davies-Bouldin Index (the average ‘similarity’ between clusters, where the similarity is a measure that compares the distance between clusters with the size of the clusters themselves. Values closer to zero indicate a better partition.)
print(f'Davies-Bouldin Index: {davies_bouldin_score(pcs_df, labels)}')

Silhouette Score(n=6): 0.36290714275758923
Variance Ratio Criterion: 621.6824513669377
Davies-Bouldin Index: 0.8250679511244968


## Affinity Propagation

In [20]:
# Initialize the Affinity Propagation model.
af = AffinityPropagation()

# Fit the model
clustering = af.fit(pcs_df)

C:\Users\ryan4\anaconda3\envs\mlenv\lib\site-packages\sklearn\cluster\_affinity_propagation.py:154: FutureWarning:

'random_state' has been introduced in 0.23. It will be set to None starting from 1.0 (renaming of 0.25) which means that results will differ at every function call. Set 'random_state' to None to silence this warning, or to 0 to keep the behavior of versions <0.23.



In [22]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the ml_housing_df and pcs_df DataFrames on the same columns.
clustered_af_df = ml_housing_income_pop_df.join(pcs_df, how='inner')

#  Add a new column, "metro_area/city" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_af_df = clustered_af_df.join(metro_name_df, how='inner')

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
# Add the predicted class columns
clustered_af_df["Class"] = clustering.labels_
clustered_af_df.head(10)

# Print the shape of the clustered_df
print(clustered_af_df.shape)
clustered_af_df.head(10)

(917, 13)


,median_listing_price,active_listing_count,median_days_on_market,new_listing_count,average_listing_price,total_listing_count,income,pop,PC 1,PC 2,PC 3,metro_area_city,Class
0,182846.428567,161.750000,81.333333,43.333333,224213.729800,162.083333,30802,28257,-0.499891,0.025396,-0.336139,"aberdeen, sd",9
1,243286.190475,336.000000,71.375000,110.000000,281836.496283,461.666667,23090,16756,-0.551533,0.103539,-0.358122,"aberdeen, wa",63
2,214853.273808,561.166667,66.416667,195.000000,247640.667450,843.166667,24529,123420,-0.186804,0.364752,-0.641400,"abilene, tx",63
3,155062.500008,151.666667,67.500000,44.333333,192856.850308,205.333333,23538,17235,-0.784764,0.575732,-0.614407,"ada, ok",21
4,169820.238100,428.416667,64.416667,123.333333,221564.868992,459.666667,20799,20600,-0.738936,0.646516,-0.590835,"adrian, mi",21
5,165456.904758,1701.500000,54.166667,798.666667,244281.999525,2889.083333,24688,197597,0.474245,0.879584,-1.022092,"akron, oh",53
6,185736.904767,410.666667,99.833333,90.666667,220033.831333,514.833333,24999,31980,-0.744040,0.334229,0.574511,"alamogordo, nm",58
7,120405.726192,545.666667,91.500000,126.666667,150925.848925,563.250000,20313,72130,-0.897690,1.032337,0.362880,"albany, ga",20
8,349433.928575,450.583333,50.375000,232.333333,448459.134575,682.500000,29664,55338,0.286068,-0.874189,-1.239458,"albany, or",0
9,297317.107150,3856.000000,83.000000,1118.666667,342436.266492,6182.666667,26440,70295,0.938669,0.205143,0.313412,"albany-schenectady-troy, ny",10


In [23]:
# Find Austin metro area 
austin_group_af = clustered_af_df.loc[clustered_af_df['metro_area_city'].str.contains("austin", case=False)]
austin_group_af

,median_listing_price,active_listing_count,median_days_on_market,new_listing_count,average_listing_price,total_listing_count,income,pop,PC 1,PC 2,PC 3,metro_area_city,Class
52,155880.357150,124.083333,60.125000,42.000000,185615.222250,177.083333,27965,25233,-0.565727,0.374262,-1.054393,"austin, mn",17
53,359709.845233,7644.250000,56.708333,3252.666667,552108.749992,10896.666667,38904,556140,4.516994,0.150833,-0.777391,"austin-round rock, tx",3


In [95]:
# Metro_area/city with austin group
metro_area_city_af = clustered_af_df.loc[clustered_af_df['Class'] == 3]
print(f'Variance Ratio Criterion: {calinski_harabasz_score(pcs_df, clustering.labels_)}')
print(f'Davies-Bouldin Index: {davies_bouldin_score(pcs_df, clustering.labels_)}')
print(f'Number of Cities: {metro_area_city_af.shape[0]}')
metro_area_city_af.head(10)

Variance Ratio Criterion: 665.7160899293718
Davies-Bouldin Index: 0.7540621556060002
Number of Cities: 5


,median_listing_price,active_listing_count,median_days_on_market,new_listing_count,average_listing_price,total_listing_count,income,pop,PC 1,PC 2,PC 3,metro_area_city,Class
53,359709.845233,7644.250000,56.708333,3252.666667,552108.749992,10896.666667,38904,556140,4.516994,0.150833,-0.777391,"austin-round rock, tx",3
56,323465.178567,9550.916667,55.833333,3853.000000,417196.827275,10743.333333,43368,249434,4.121117,0.189771,-0.838788,"baltimore-columbia-towson, md",3
222,504546.892858,7635.750000,41.291667,4697.666667,670959.989258,13907.416667,37628,421478,5.232733,-0.379517,-0.983159,"denver-aurora-lakewood, co",3
546,358345.476192,10025.416667,46.583333,5232.333333,469037.625650,15911.500000,37062,274215,4.998096,0.740847,-0.705300,"minneapolis-st. paul-bloomington, mn-wi",3
582,368854.047617,7766.333333,39.750000,3958.666667,537439.706383,10374.916667,38063,300272,4.121728,0.027575,-1.252343,"nashville-davidson--murfreesboro--franklin, tn",3


In [ ]:
# Export Affinity Propagation Cluster to .CSV
# metro_area_city_af.to_csv(r'../Resources/Affinity_output.csv', index = True)

### Affinity Propagation Plots

In [118]:
# Creating a 3D-Scatter with the PCA data and the clusters
# Plot
fig = px.scatter_3d(clustered_af_df, x="PC 1", y="PC 2", z="PC 3", color="Class", symbol="Class",size="average_listing_price", size_max=25, width=800, hover_name="metro_area_city", hover_data=["average_listing_price"])
fig.update_layout(legend=dict(x=0,y=1))
fig.show()

In [26]:
# 2D scatter plot
#plt.scatter(clustered_af_df[:,0], [:,1], c=clustering.labels_, cmap='rainbow', alpha=0.7, edgecolors='b')
clustered_af_df.hvplot.scatter(
    x="PC 1",
    y="PC 2",
    hover_cols=["metro_area_city"],
    by="Class",
)

:NdOverlay   [Class]
   :Scatter   [PC 1]   (PC 2,metro_area_city)

## Hierarchical Agglomerative Clustering

In [126]:
# Create the dendrogram
fig = ff.create_dendrogram(pcs_df, color_threshold=4.5)
fig.update_layout(width=800, height=1000)
fig.show()

In [127]:
# # Initialize the Hierarchical Agglomerative Clustering model.
agg = AgglomerativeClustering(n_clusters=15)

# fit the model
hac = agg.fit(pcs_df)

In [128]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the ml_housing_df and pcs_df DataFrames on the same columns.
clustered_hac_df = ml_housing_income_pop_df.join(pcs_df, how='inner')

#  Add a new column, "metro_area/city" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_hac_df = clustered_hac_df.join(metro_name_df, how='inner')

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
# Add the predicted class columns
clustered_hac_df["Class"] = hac.labels_
clustered_hac_df.head(10)

# Print the shape of the clustered_df
print(clustered_hac_df.shape)
clustered_hac_df.head(10)

(917, 13)


,median_listing_price,active_listing_count,median_days_on_market,new_listing_count,average_listing_price,total_listing_count,income,pop,PC 1,PC 2,PC 3,metro_area_city,Class
0,182846.428567,161.750000,81.333333,43.333333,224213.729800,162.083333,30802,28257,-0.499891,0.025396,-0.336139,"aberdeen, sd",8
1,243286.190475,336.000000,71.375000,110.000000,281836.496283,461.666667,23090,16756,-0.551533,0.103539,-0.358122,"aberdeen, wa",5
2,214853.273808,561.166667,66.416667,195.000000,247640.667450,843.166667,24529,123420,-0.186804,0.364752,-0.641400,"abilene, tx",8
3,155062.500008,151.666667,67.500000,44.333333,192856.850308,205.333333,23538,17235,-0.784764,0.575732,-0.614407,"ada, ok",14
4,169820.238100,428.416667,64.416667,123.333333,221564.868992,459.666667,20799,20600,-0.738936,0.646516,-0.590835,"adrian, mi",14
5,165456.904758,1701.500000,54.166667,798.666667,244281.999525,2889.083333,24688,197597,0.474245,0.879584,-1.022092,"akron, oh",8
6,185736.904767,410.666667,99.833333,90.666667,220033.831333,514.833333,24999,31980,-0.744040,0.334229,0.574511,"alamogordo, nm",5
7,120405.726192,545.666667,91.500000,126.666667,150925.848925,563.250000,20313,72130,-0.897690,1.032337,0.362880,"albany, ga",6
8,349433.928575,450.583333,50.375000,232.333333,448459.134575,682.500000,29664,55338,0.286068,-0.874189,-1.239458,"albany, or",9
9,297317.107150,3856.000000,83.000000,1118.666667,342436.266492,6182.666667,26440,70295,0.938669,0.205143,0.313412,"albany-schenectady-troy, ny",3


In [129]:
# Find Austin metro area 
austin_group_hac = clustered_hac_df.loc[clustered_hac_df['metro_area_city'].str.contains("austin", case=False)]
austin_group_hac

,median_listing_price,active_listing_count,median_days_on_market,new_listing_count,average_listing_price,total_listing_count,income,pop,PC 1,PC 2,PC 3,metro_area_city,Class
52,155880.357150,124.083333,60.125000,42.000000,185615.222250,177.083333,27965,25233,-0.565727,0.374262,-1.054393,"austin, mn",8
53,359709.845233,7644.250000,56.708333,3252.666667,552108.749992,10896.666667,38904,556140,4.516994,0.150833,-0.777391,"austin-round rock, tx",3


In [130]:
# Metro_area/city with austin group
metro_area_city_hac = clustered_hac_df.loc[clustered_hac_df['Class'] == 3]
print(f'Variance Ratio Criterion: {calinski_harabasz_score(pcs_df, hac.labels_)}')
print(f'Davies-Bouldin Index: {davies_bouldin_score(pcs_df, hac.labels_)}')
print(f'Number of Cities: {metro_area_city_hac.shape[0]}')
metro_area_city_hac.head(10)

Variance Ratio Criterion: 596.9817937264071
Davies-Bouldin Index: 0.8394336901884958
Number of Cities: 42


,median_listing_price,active_listing_count,median_days_on_market,new_listing_count,average_listing_price,total_listing_count,income,pop,PC 1,PC 2,PC 3,metro_area_city,Class
9,297317.107150,3856.000000,83.000000,1118.666667,342436.266492,6182.666667,26440,70295,0.938669,0.205143,0.313412,"albany-schenectady-troy, ny",3
13,268651.244050,2470.083333,54.958333,1104.333333,350132.412267,4352.666667,30403,560513,2.158623,0.496140,-1.161531,"albuquerque, nm",3
53,359709.845233,7644.250000,56.708333,3252.666667,552108.749992,10896.666667,38904,556140,4.516994,0.150833,-0.777391,"austin-round rock, tx",3
56,323465.178567,9550.916667,55.833333,3853.000000,417196.827275,10743.333333,43368,249434,4.121117,0.189771,-0.838788,"baltimore-columbia-towson, md",3
66,244560.416667,4171.166667,82.291667,1043.666667,310885.241750,4531.416667,28491,220236,1.168134,0.466301,0.087839,"baton rouge, la",3
88,249661.928575,4097.250000,67.833333,1435.666667,323360.152908,5488.333333,35005,147585,1.519239,0.120473,-0.600326,"birmingham-hoover, al",3
136,303677.232142,9477.416667,86.375000,2283.333333,509095.152358,12299.750000,29980,140799,2.840199,0.459692,0.808653,"cape coral-fort myers, fl",3
154,341191.744050,8189.833333,57.833333,3566.333333,456165.700833,12507.500000,32563,353107,3.938183,0.660109,-0.416211,"charlotte-concord-gastonia, nc-sc",3
161,270155.732142,5471.500000,54.166667,2539.000000,354225.385358,9727.250000,30531,303940,2.724807,0.790292,-0.769175,"cincinnati, oh-ky-in",3
169,195869.404767,5874.916667,63.166667,2358.666667,285278.250925,9608.916667,22833,217383,1.967354,1.500810,-0.189533,"cleveland-elyria, oh",3


In [ ]:
# Export Agglomerative Cluster to .CSV
# metro_area_city_hac.to_csv(r'../Resources/Agglomerative_output.csv', index = True)

### Hierarchical Agglomerative Clustering Plots

In [132]:
# Creating a 3D-Scatter with the PCA data and the clusters
# Plot
fig = px.scatter_3d(clustered_hac_df, x="PC 1", y="PC 2", z="PC 3", color="Class", symbol="Class",size="average_listing_price", size_max=25, width=800, hover_name="metro_area_city", hover_data=["average_listing_price"])
fig.update_layout(legend=dict(x=0,y=1))
fig.show()

In [133]:
# 2D scatter plot
clustered_hac_df.hvplot.scatter(
    x="PC 1",
    y="PC 2",
    hover_cols=["metro_area_city"],
    by="Class",
)

:NdOverlay   [Class]
   :Scatter   [PC 1]   (PC 2,metro_area_city)

## Connect to Local Postgres - Only use if AWS does not work** 

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# follows django database settings format, replace with your own settings
DATABASES = {
    'production':{
        'NAME': 'sleeper_cities',
        'USER': 'postgres',
        'PASSWORD': 'Staypositive1504!',
        'HOST': 'localhost',
        'PORT': 5432,
    },
}

# choose the database to use
db = DATABASES['production']

# construct an engine connection string
engine_string = "postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}".format(
    user = db['USER'],
    password = db['PASSWORD'],
    host = db['HOST'],
    port = db['PORT'],
    database = db['NAME'],
)

# create sqlalchemy engine
engine = create_engine(engine_string)

# read a table from database into pandas dataframe, replace "tablename" with your table name
local_df = pd.read_sql_table('housing_data',engine)
df.head()